In [211]:
import requests 
import json 
import pprint
from datetime import date


SyntaxError: invalid syntax (<ipython-input-211-2c030547e70a>, line 5)

In [257]:
class Report: 
    def __init__(self):
        
        self.url ="https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest" #link per i report
        self.params = {
            'start': '1', #inizia dalla prima cripto in classifica
            'limit': '100',  #finisce con la 100'esima cripto in classifca
            'convert': "USD"  #valuta delle cripto -> dolalto 
        }
        
        self.headers = {
            'Accepts': 'application/json', #formato della risposta 
            'X-CMC_PRO_API_KEY': 'e5c4253e-a74e-4274-84a0-9ed17d09b04f' #chiave per poter scaricare i dati 
        }
        
    def fetchCurrentData(self): 
        r = requests.get(url = self.url, headers = self.headers, params = self.params).json() #fa la richieste per i dati 
        return r['data'] #restituisce i dati in formato json 


In [258]:
report = Report()

# currencies = report.fetchCurrentData()
# with open("output.json", "w") as outfile: 
#     json.dump(currencies, outfile, indent = 4)

data = json.load(open("old_data.json"))


In [259]:
def max_volume(data): #trova la valuta con maggiore volume delle ultime 24h
    volumes = {v['quote']['USD']['volume_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    return volumes[max(volumes)] #ritorna nome della cripto con volume maggiore


In [260]:
def best_and_worst_criptos(data): #trova le migliori e le peggiori 10 criptovalute (per incremento in percentuale nelle ultime 24 ore)

    volumes = { v['quote']['USD']['percent_change_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    sorted_volumes = sorted(volumes) #lista ordinata in base alle chiavi del dizionario
    best_volumes = sorted_volumes[:10]  #lista con i volumi migliori
    worst_volumes = sorted_volumes[-10:] #lista con i volumi peggiori
    
    best = []
    worst = []
    
    for v,name in volumes.items():  #ciclo sul dizionario 
        if v in best_volumes:  #se la chiave è nella lista dei migiori 
            best.append(name)  #salvo il nome della cripto 
        if v in worst_volumes: 
            worst.append(name)
            
    return best, worst





In [273]:
def first_20(data): #trova la quantità di denaro necessaria per acquistare una unità di ciascuna delle prime 20 criptovalute
    volumes = {v['name']: v['quote']['USD']['price'] for v in data[:20]} #dizionario con chiave nome e valore
    return volumes



{'Bitcoin': 9153.32867575, 'Ethereum': 230.292527905, 'Tether': 1.00117650284, 'XRP': 0.182749274109, 'Bitcoin Cash': 230.036873861, 'Bitcoin SV': 168.804520428, 'Litecoin': 43.2094013667, 'Binance Coin': 15.7177865959, 'EOS': 2.47187407701, 'Cardano': 0.0804754056946, 'Crypto.com Coin': 0.114398090749, 'Tezos': 2.47372032749, 'Chainlink': 4.69943909929, 'Stellar': 0.0671848251789, 'UNUS SED LEO': 1.22562119267, 'Monero': 63.8095841541, 'TRON': 0.015798389452, 'USD Coin': 1.00185668617, 'Huobi Token': 4.21021792924, 'Neo': 10.2092286668}


In [268]:
def find_criptos(data, volume): 
    #trova la quantità di denaro necessaria per aqustare una unità di tutte le cripto
    #il cui volume delle ultime 24 h sia superiore a 'volume'
    
    volumes = {v['name']: v['quote']['USD']['price'] for v in data
                if v['quote']['USD']['volume_24h'] > volume}
    
    return zip(volumes)


In [263]:
#trova la precentuale di guadagno o perdita che ho se comprassi una unità di ciascuna
#delle prime 20 criptovalute il giorno prima (ipotizzando che la classifica non sia cambiata)
def find_percentage(new_data, old_data):
    new_20 = first_20(new_data)
    old_20 = first_20(old_data)
    
    #creo dizionario con nome e precentuale du guadagno o peridata
    r = {f : (new_20[f] - old_20[i]) / old_20[f] *100 for f,i in zip(new_20,old_20)}
    return r
    

In [288]:
today = date.today() #data di oggi 

max_volume_cripto = max_volume (data)
bests, worsts = best_and_worst_criptos(data) 
f_20 = first_20(data)
criptos = find_criptos(data, 76000000)
percentage = find_percentage(new_data, old_data)

result = {'max_volume': max_volume_cripto, 
            'bests': bests,
            'worst': worsts,
            'first_20_criptos': f_20,
             'criptos': criptos,
            'percentage': percentage}

print(result)
type(result)
with open(f"{today}.json", "w") as outfile: 
     json.dump(bests, outfile, indent = 4)

{'max_volume': 'Tether', 'bests': ['Algorand', 'Aave', 'Ren', 'Verge', 'Komodo', 'SwissBorg', 'Bancor', 'Hive', 'MCO', 'WAX'], 'worst': ['UNUS SED LEO', 'Compound', '0x', 'Synthetix Network Token', 'Nexo', 'ZB Token', 'Quant', 'Loopring', 'Celsius', 'BitShares'], 'first_20_criptos': {'Bitcoin': 9153.32867575, 'Ethereum': 230.292527905, 'Tether': 1.00117650284, 'XRP': 0.182749274109, 'Bitcoin Cash': 230.036873861, 'Bitcoin SV': 168.804520428, 'Litecoin': 43.2094013667, 'Binance Coin': 15.7177865959, 'EOS': 2.47187407701, 'Cardano': 0.0804754056946, 'Crypto.com Coin': 0.114398090749, 'Tezos': 2.47372032749, 'Chainlink': 4.69943909929, 'Stellar': 0.0671848251789, 'UNUS SED LEO': 1.22562119267, 'Monero': 63.8095841541, 'TRON': 0.015798389452, 'USD Coin': 1.00185668617, 'Huobi Token': 4.21021792924, 'Neo': 10.2092286668}, 'criptos': <zip object at 0x06FAED68>, 'percentage': {'Bitcoin': 7.068466815947549, 'Ethereum': -13.02690984936149, 'Tether': 0.0, 'XRP': 0.0, 'Bitcoin Cash': 0.0, 'Bitcoi